# Отчёт о решении контеста

### Формулировка задачи

Данные представлены в признаковом описании, где признаками служат:
* Num - номер в таблице (целое число; в регрессии не используется)
* y - показатель спроса (целое число) - используется в качестве таргета
* year - год (целое число от 2012 до 2015)
* week - номер недели в году (целое число от 1 до 53)
* shift - смена (целое число от 1 до 3, каждую неделю новая смена)
* item_id - номер товара (целое число; в регрессии не используется, поскольку не имеет количественного смысла)
* f1, f2, ..., f60 -  предподсчитанные средние значения продаж по разным срезам (целые числа)
Необходимо по данной таблице признаков предсказать столбец y - величину спроса.
Показателем качества предсказания служит $SMAPE = \frac{100\%}{n} \sum_{i=1}^{n} \frac{2|\hat{y}_i - y_i|}{|\hat{y}_i| + |y_i|}$

### Описание решения
Поскольку в таблице признаков представлены только числовые значения (номер и идентификатор товара, как не имеющие количественного смысла, мы исключаем из таблицы), можно провести регрессию по всем признакам. Столбец 'y' будет служить таргетом для регрессии.
В качестве основного алгоритма используются ансамбли решающих деревьев, поскольку данные не позволяют построить адекватную интерпретируемую линейную модель. 

### Подходы к решению
Очевидно, "сырое" решающее дерево не даёт удовлетворительного результата. Для решения использовались различные ансамбли решающих деревьев: GradientBoostingRegressor и RandomForestRegressor из библиотеки sklearn и XGBRegressor из библиотеки XGBoost. Среди перечисленных последний лидирует по качеству на тестовой выборке с большим отрывом: MAE порядка $3 \times 10^5$ против $5 \times 10^5$ и более. Из параметров регрессии сначала настраивалось n_estimators - число деревьев в ансамбле (из соображений возможности вычислить за разумное время и простых оценок качества, полученных из графиков $MAE = f(n\_estimators)$. Затем перебором по нескольким небольшим значениям и сравнением MAE были настроены параметры max_depth - максимальная глубина дерева и min_child_weight - ограничение по количеству элементов обучающей выборки в узле дерева. Эти параметры помогают избежать переобучения. MAE оценивалось по 3-fold кроссвалидации. Следует отметить, что рост MAE не всегда приводит к росту SMAPE, но в целом для грубой оценки можно довольствоваться MAE, поскольку обе метрики определяются модулями разности предсказания и истинного значения.

### Код
Ниже приведён код решения.

In [4]:
import pandas as pd

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

X = train.drop(['Num','y'], axis=1)
y = train['y']

#from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import numpy as np

n = 80
max_depth = 12
min_child_weight = 2

model = XGBRegressor(n_estimators=n, max_depth=max_depth,
                     min_child_weight=min_child_weight)
    
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))

score_mae = cross_val_score(model, X, y, scoring =  'neg_mean_absolute_error', cv = 3).mean()
print(score_mae)

sample_submission['y'] = preds
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)


# Дополнительные вопросы

1. sample_submission.tsv является примером предсказания на тестовой выборке. В нём на каждый набор признаков отвечают одним и тем же значением - средним в столбце y обучающей выборки (файла train.csv). Например, в точности такое же число, 198575.912031, является результатом вызова кода:

In [2]:
import pandas as pd
train = pd.read_csv("train.tsv")
print(train['y'].mean())

198575.912031
